In [7]:
# The Guardian API Extract

import csv
import requests

api_key = " "  # API key here
endpoint = "http://content.guardianapis.com/search"

# Enter Search or Query words
search_terms = '("Refugees" OR "Migration" OR "Immigration" OR "Refugee" OR "Asylum Seeker" OR "Asylum Seekers" OR "Migrants" OR "Migrant" OR "Immigrant" OR "Immigrants")'

params = {
    "q": search_terms,  # Pass the search terms as the query
    "from-date": "2021-01-01",
    "api-key": api_key,  # Pass the api_key as a string
    "page-size": 100,  # The number of articles per page
    "page": 1,  # The starting page
    "show-fields": "body"  # Include the article body in the API response
}

filename = '/Users/henryasiamah/Desktop/work/guardian_news.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'publication_date', 'sectionName', 'webTitle', 'body'])

    while True:
        response = requests.get(endpoint, params=params)
        data = response.json()
        
        if 'results' not in data['response']:
            break
        
        articles = data['response']['results']
        if not articles:
            break

        for article in articles:
            id_parts = article['id'].split('/')
            if len(id_parts) == 5:
                publication_date = '/'.join(id_parts[1:4])
            else:
                publication_date = '/'.join(id_parts[1:5])
            writer.writerow([article['id'], publication_date, article['sectionName'], article['webTitle'], article.get('fields', {}).get('body', '')])

        params['page'] += 1

In [8]:
# The extracted date column was in an odd format, therefore we are adding another column 'publication_date_fixed to have a stradard format of it
import datetime
import sys 

csv.field_size_limit(sys.maxsize)

filename = '/Users/henryasiamah/Desktop/work/guardian_news.csv'
new_filename = '/Users/henryasiamah/Desktop/work/newone_guardian_news.csv'

with open(filename, 'r') as file, open(new_filename, 'w', newline='') as new_file:
    reader = csv.reader(file)
    writer = csv.writer(new_file)

    headers = next(reader)
    headers.append('publication_date_fixed')
    writer.writerow(headers)

    for row in reader:
        id = row[0]
        id_parts = id.split('/')
        if len(id_parts) >= 4:
            date_string = f"{id_parts[-4]} {id_parts[-3]} {id_parts[-2]}"
            date_format = "%Y %b %d"
            publication_date_fixed = datetime.datetime.strptime(date_string, date_format).strftime("%Y-%m-%d")
            row.append(publication_date_fixed)
        else:
            row.append('')
        writer.writerow(row)

In [9]:
import pandas as pd
import numpy as np

In [10]:
file = pd.read_csv("/Users/henryasiamah/Desktop/work/newone_guardian_news.csv")
file.head()

,id,publication_date,sectionName,webTitle,body,publication_date_fixed
0,uk-news/2024/oct/14/doctors-say-uks-immigratio...,2024/oct/14,UK news,Doctors say UK’s immigration system risks ‘re-...,<p>Top doctors have identified Britain’s immig...,2024-10-14
1,uk-news/article/2024/aug/10/messages-of-welcom...,article/2024/aug/10,UK news,Messages of welcome to be delivered to refugee...,<p>A Brighton-based organisation has promised ...,2024-08-10
2,world/2024/oct/11/echr-ruling-for-cyprus-asylu...,2024/oct/11,World news,ECHR ruling for Cyprus asylum seekers may embo...,<p>A ruling by the European court of human rig...,2024-10-11
3,world/2024/oct/18/blow-to-melonis-albania-deal...,2024/oct/18,World news,Blow to Meloni’s Albania deal as court orders ...,<p>The last 12 asylum seekers being held in a ...,2024-10-18
4,uk-news/2024/sep/15/starmer-under-pressure-to-...,2024/sep/15,UK news,Starmer under pressure to distance UK from Ita...,<p>Keir Starmer is under pressure from Labour ...,2024-09-15


In [11]:
file.shape

(33400, 6)

In [12]:
file.tail()

,id,publication_date,sectionName,webTitle,body,publication_date_fixed
33395,world/live/2020/oct/03/coronavirus-live-news-a...,live/2020/oct/03,World news,Italy infections spike; Madrid tightens lockdo...,"<div id=""block-5f7929708f08abd321a6c384"" class...",2020-10-03
33396,australia-news/live/2020/oct/09/coronavirus-au...,live/2020/oct/09,Australia news,NSW issues new alert as Victoria reports 11 ne...,"<div id=""block-5f80147e8f089f7174162500"" class...",2020-10-09
33397,us-news/live/2020/jan/23/donald-trump-impeachm...,live/2020/jan/23,US news,Impeachment trial: Democrats cry hypocrisy as ...,"<div id=""block-5e2af9038f08a6950eb483c9"" class...",2020-01-23
33398,world/live/2020/nov/19/coronavirus-live-news-u...,live/2020/nov/19,World news,Italy reports 653 new deaths – as it happened,"<div id=""block-5fb702e28f08b4f3c5676e7e"" class...",2020-11-19
33399,uk-news/live/2020/jun/12/uk-coronavirus-live-e...,live/2020/jun/12,Politics,UK coronavirus: Grant Shapps denies chief nurs...,"<div id=""block-5ee3bd7f8f08e3e3b74b51df"" class...",2020-06-12
